## Undersøkelse og forberedelse av data

Valg av forklaringvariabler basert på relevans, etikk og data inspection. Eventuelt egenkomponerte forklaringsvariabler.

Før vi kan begynne med modelleringen er vi nødt til å undersøke dataene grundig. Vi må få en oversikt over:
* Størrelsen
* Fordelingen på fasiten vår
* Mangler vi data?
* Finner vi noe rart?
* Hva er kategorisk og hva er numerisk?
* Hva må mappes og hva må ordnes?
* Ser vi noen sammenhenger som utgjør nye variabler?
* Enkoding

#### Vi starter med å lese inn datasettet til en pandas dataframe

In [26]:
import pandas as pd
df = pd.read_csv('data/fraud.csv')

#### La oss ta en første kikk på dataene, for få oversikt over hva vi skal jobbe med

In [27]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,11555,11556,11557,11558,11559,11560,11561,11562,11563,11564
ID,1068,2945,6883,6121,15197,7462,7104,7882,9432,8667,...,1052,3800,13073,9099,2656,4239,8900,11245,8374,2226
Month,Nov,Jun,Jan,Nov,Oct,Sep,Feb,Dec,Oct,Dec,...,Mar,May,Oct,May,Nov,Jan,Apr,Aug,May,Feb
WeekOfMonth,5,5,3,4,3,2,2,3,2,3,...,5,3,4,4,3,4,3,1,3,3
DayOfWeek,Monday,Thursday,Friday,Saturday,Saturday,Thursday,Wednesday,Thursday,Saturday,Saturday,...,Tuesday,Thursday,Monday,Tuesday,Saturday,Saturday,Monday,Sunday,Sunday,Monday
Make,Accura,Chevrolet,Accura,Mazda,Mazda,Toyota,Toyota,Mazda,Honda,Mazda,...,Chevrolet,Chevrolet,Mazda,Honda,Honda,Ford,Honda,Toyota,Toyota,Pontiac
AccidentArea,Urban,Urban,Urban,Urban,Rural,Urban,Urban,Urban,Urban,Urban,...,Urban,Urban,Rural,Urban,Urban,Rural,Urban,Rural,Urban,Urban
DayOfWeekClaimed,Thursday,Friday,Thursday,Monday,Monday,Tuesday,Tuesday,Thursday,Tuesday,Wednesday,...,Friday,Friday,Wednesday,Tuesday,Monday,Tuesday,Monday,Tuesday,Monday,Thursday
MonthClaimed,Dec,Jul,Feb,Nov,Oct,Sep,Feb,Dec,Oct,Dec,...,Apr,May,Oct,May,Nov,Jan,Apr,Aug,May,Mar
WeekOfMonthClaimed,1,5,3,5,3,2,3,3,2,4,...,1,3,4,4,4,5,3,2,3,1
Sex,Male,Male,Male,Male,Male,Male,Female,Female,Male,Male,...,Male,Male,Female,Male,Male,Male,Male,Male,Female,Female


In [ ]:
df["Sex"].value_counts()

#### Prosentvis fordeling av fasit

In [ ]:
total_fraud=(df.FraudFound_P.value_counts()[1]+df.FraudFound_P.value_counts()[0])
not_fraud=df.FraudFound_P.value_counts()[0]
fraud=df.FraudFound_P.value_counts()[1]
prct_not_fraud=not_fraud/total_fraud*100
prct_fraud=fraud/total_fraud*100
print(f'The data consists of {df.shape[1]} features and {len(df)} observations.')
print(f"{round(prct_not_fraud)}% of the obervations are not marked as fraud")
print(f"{round(prct_fraud)}% of the observations are marked as fraud")

#### Lister alle forklaringsvariabler med korresponderende datatyper 

In [ ]:
df.dtypes

#### Lister unike verdier for hver forklaringsvariabel

In [ ]:
from pprint import pprint
for col in df:
    print(f"{col}: {df[col].unique()}")       

#### Vi deler forklaringsvariablene inn i kategoriske og numeriske variabler
Ved hjelp av oversikten over kan vi avgjøre hvilke forklaringsvariabler som skal være kategoriske og hvilke som skal være numeriske. Om vi bestemmer at en forklaringsvariabel skal være kategorisk, kan det allikevel være nyttig å ordne eller sortere verdiene. Det kan hjelpe tremodellen vår til å gjøre bedre og mer effektive valg. Det kan også være nyttig å lage en ny variabel basert på en vi allerede har. For eksempel fra måned til sesong.

#### Vi legger til en ny forkalringsvariabel "Season" ved å mappe Month til Season

In [ ]:
season_mapping = {
    "Jan":"Winter",
    "Feb":"Winter",
    "Mar":"Spring",
    "Apr":"Spring",
    "May":"Spring",
    "Jun":"Summer",
    "Jul":"Summer",
    "Aug":"Summer",
    "Sep":"Fall",
    "Oct":"Fall",
    "Nov":"Winter",
    "Dec":"Winter"
}
df["Season"] = df["Month"].map(season_mapping)
print(df["Season"].unique())

#### Mapping av den kategoriske variabelen Month fra string til en ordnet tallrekke av typen int

In [ ]:
month_mapping = {
    "Jan":1,
    "Feb":2,
    "Mar":3,
    "Apr":4,
    "May":5,
    "Jun":6,
    "Jul":7,
    "Aug":8,
    "Sep":9,
    "Oct":10,
    "Nov":11,
    "Dec":12
}
df=df.replace({"Month": month_mapping})
print(df["Month"].unique())

#### Vi mapper den numeriske variabelen AddressChange_Claim til en ordnet tallrekke


In [ ]:
ageofvehicle_mapping = {
    "new": 1,
    "2 years": 2,
    "3 years": 3,
    "4 years": 4,
    "5 years": 5,
    "6 years": 6,
    "7 years": 7,
    "more than 7": 8
}
df=df.replace({"AgeOfVehicle": ageofvehicle_mapping})
print(df["AgeOfVehicle"].unique())

#### Her skriver du inn alle forklaringsvariablene som du vurderer å være kategoriske

Det er lagt inn et par eksempler, men det er nok fler!

In [ ]:
categorical_features=['Month', 'Sex']

#### Vi sjekker om det er duplikater

In [ ]:
df.duplicated().any()

#### Vi sjekker om det er "NA"-verdier i datasettet

NA er en paraplybetegnelse for spesielle typer som er ment å symbolisere en manglende verdi, eller en feilverdi. Noen eksempler på dette er "None", "Null" og "Not a Number"(i en numerisk variabel).

In [ ]:
df.isna().any()

#### Er det noen 0-verdier der det ikke bør være det?

In [ ]:
for col in df:
    if df[col].dtype == int:
        count = (df[col] == 0).sum()
    else:
        count = (df[col] == '0').sum()
    if count > 0:
        print(f'{col} {count}')

Ja, vi fant ut at Age-variabelen har så mange som 320 0-verdier, her er det noe som ikke stemmer.
Vi må finne ut hva vi skal mappe denne til. Vi kan enten finne en annen verdi/mean-verdi basert på informasjon fra de andre variablene. Eller vi kan beholde den som 0. I en linær modell er nok en mean verdi viktigere å sette inn enn i en tremodell.

In [ ]:
df["Age"].loc[df["Age"] == 0] = 16

In [ ]:
print(df["Age"].unique())

Herfølger eksempelkode for å plotte distribusjonen av verdiene i en kontinuerlig inputvariabel:

In [ ]:
import seaborn as sns
import scipy.stats as stats

skew = df["Age"].skew()
ax = sns.distplot(df["Age"], fit = stats.norm)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
df["Sex"].value_counts()

In [16]:
from sklearn.preprocessing import LabelEncoder

In [25]:
df["Sex"].value_counts()

Male      9535
Female    1798
16         232
Name: Sex, dtype: int64

#### Encoding vil bare fungere dersom du har mappet alle de numeriske forklaringsvariablene riktig. 

In [17]:
df[categorical_features] = df[categorical_features].apply(LabelEncoder().fit_transform)

df.to_parquet('data/fraud_processed.pq')  # Lagre prosessert data ut til en fil i parquet-format, som kan leses inn i den andre notebooken

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['int', 'str']